In [1]:
import cv2
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output

ModuleNotFoundError: No module named 'cv2'

In [2]:
def detect_face(frame):
    detector = cv2.CascadeClassifier("xml/frontal_face.xml")
    faces = detector.detectMultiScale(frame,1.2,5)
    return faces

In [3]:
def cut_faces(image, faces_coord):
    faces = []
    for (x, y, w, h) in faces_coord:
        faces.append(image[y: y + h, x : x + w ])
    return faces

In [4]:
def normalize_intensity(images):
    images_norm = []
    for image in images:
        images_norm.append(cv2.equalizeHist(image))
    return images_norm

In [5]:
def resize(images,size=(47,62)):
    image_resize = []
    
    for image in images:
        if image.shape < size:
            img_size = cv2.resize(image,size,interpolation=cv2.INTER_CUBIC)
        else:
            img_size = cv2.resize(image,size,interpolation=cv2.INTER_AREA)
        image_resize.append(img_size)
        
    return image_resize

In [6]:
def normalize_faces(frame, faces_coord):
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = cut_faces(gray_frame, faces_coord)
    faces = normalize_intensity(faces)
    
    faces = resize(faces)
    return faces

In [7]:
def plot_show(image,title=""):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.title(title)
    plt.imshow(image,cmap="Greys_r")
    plt.show()

In [8]:
def draw_rectangle(image, coords):
    for (x, y, w, h) in coords:
        
        cv2.rectangle(image, (x , y), (x + w , y + h), (0,0,255),2)

In [9]:
cam = cv2.VideoCapture(0)
#cv2.namedWindow("Face", cv2.WINDOW_AUTOSIZE)
try:
    os.mkdir("people")
except:
    pass
folder = "people/"+input('Person:').lower()
if not os.path.exists(folder):
    os.mkdir(folder)
    flag_start_capturing = False
    sample=1
    cv2.namedWindow("Face", cv2.WINDOW_NORMAL)
    while True:
        ret,frame = cam.read()
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces_coord = detect_face(gray)
        if len(faces_coord):
            faces = normalize_faces(frame,faces_coord)
            #faces = normalize_intensity(faces)
            cv2.imwrite(folder + '/' + str(sample)+'.jpg',faces[0])
            plot_show(faces[0],"Image saved:"+str(sample))
            clear_output(wait=True)
            if flag_start_capturing == True:
                sample += 1
        draw_rectangle(frame,faces_coord)
        cv2.imshow('Face',frame)
        keypress=cv2.waitKey(1)
        if keypress == ord('c'):
            if flag_start_capturing == False:
                flag_start_capturing = True
        if(sample >500 or keypress == ord('q')):
            break
    cam.release()
    cv2.destroyAllWindows()
else:
    print ("This name already exists.")

NameError: name 'cv2' is not defined

In [10]:
def collect_dataset():
    images = []
    labels = []
    labels_dic = {}
   
    people = [person for person in os.listdir("people/")]
   
    for i, person in enumerate(people):
        labels_dic[i] = person
        for image in os.listdir("people/" + person):
            if image.endswith('.jpg'):
                images.append(cv2.imread("people/" + person + '/' + image, 0))
                labels.append(i)
    return (images, np.array(labels), labels_dic)

In [11]:
images, labels, labels_dic = collect_dataset()

NameError: name 'os' is not defined

In [12]:
print (len(images))
print (labels_dic)

NameError: name 'images' is not defined

In [41]:
images

[array([[ 94, 107, 112, ..., 235, 240, 244],
        [102, 115, 110, ..., 231, 237, 239],
        [113, 121, 114, ..., 232, 236, 235],
        ...,
        [255, 254, 251, ..., 248, 249, 251],
        [251, 255, 255, ..., 248, 252, 251],
        [255, 252, 241, ..., 252, 253, 249]], dtype=uint8),
 array([[107,  97, 103, ..., 230, 230, 238],
        [ 97,  98,  93, ..., 226, 237, 234],
        [ 91,  95, 106, ..., 219, 230, 237],
        ...,
        [255, 251, 254, ..., 237, 247, 246],
        [255, 252, 253, ..., 224, 244, 249],
        [255, 254, 253, ..., 220, 242, 248]], dtype=uint8),
 array([[ 99,  94,  93, ..., 229, 233, 231],
        [ 88, 101,  97, ..., 227, 228, 235],
        [ 95, 106,  93, ..., 206, 229, 230],
        ...,
        [253, 254, 254, ..., 219, 241, 248],
        [254, 254, 255, ..., 203, 230, 249],
        [255, 252, 254, ..., 199, 228, 244]], dtype=uint8),
 array([[ 97,  93, 101, ..., 230, 233, 233],
        [107,  92,  98, ..., 228, 231, 231],
        [100, 10

In [42]:
images[100].shape

(62, 47)

In [43]:
X_train=np.asarray(images)

In [44]:
type(X_train)
X_train.shape

(435, 62, 47)

In [45]:
train=X_train.reshape(len(X_train),-1)

In [46]:
train

array([[ 94, 107, 112, ..., 252, 253, 249],
       [107,  97, 103, ..., 220, 242, 248],
       [ 99,  94,  93, ..., 199, 228, 244],
       ...,
       [177, 145, 107, ..., 205, 207, 205],
       [197, 177, 142, ..., 204, 204, 200],
       [165, 118,  95, ..., 206, 205, 206]], dtype=uint8)

In [47]:
train.shape

(435, 2914)

In [48]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [49]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(train.astype(np.float64))

In [50]:
pca1 = PCA(n_components=.97)
new_train=pca1.fit_transform(X_train_sc)

In [51]:
new_train.shape

(435, 139)

In [52]:
1-.97

0.030000000000000027

In [53]:
2914*0.03/100


0.8742

In [54]:
pca1.n_components_

139

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,KFold

In [56]:
param = {'C':[.001,.01,.1,1,10],'penalty':['l2','l1']}

In [57]:
gd = GridSearchCV(LogisticRegression(),param_grid=param,cv=10,scoring='accuracy')

In [58]:
gd.fit(new_train,labels)

GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10], 'penalty': ['l2', 'l1']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [59]:
gd.best_score_

1.0

In [60]:
gd.best_params_

{'C': 1, 'penalty': 'l1'}

In [61]:
clf2 = gd.best_estimator_

In [62]:
filename = 'log_face.pkl'
f=open(filename, 'wb')
pickle.dump(clf2,f)
 

f.close()

In [63]:
filename = 'log_face.pkl'
log1 = pickle.load(open(filename, 'rb'))

In [64]:
kf=KFold(n_splits=5,shuffle=True)

In [65]:
param_grid = {'C':[.0001,.001,.01,.1,1,10]}

In [66]:
gs_svc = GridSearchCV(SVC(kernel='linear',probability=True),param_grid=param_grid,cv=kf,scoring='accuracy')

In [67]:
gs_svc.fit(new_train,labels)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [68]:
gs_svc.best_score_

1.0

In [69]:
gs_svc.best_params_

{'C': 0.0001}

In [70]:
clf_svc=gs_svc.best_estimator_

In [71]:
clf_svc

SVC(C=0.0001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [72]:
import pickle

In [73]:
filename = 'svc_linear_face.pkl'
f=open(filename, 'wb')
pickle.dump(clf_svc,f)
 

f.close()


In [74]:
filename = 'svc_linear_face.pkl'
svc1 = pickle.load(open(filename, 'rb'))

In [75]:
svc1

SVC(C=0.0001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [76]:
cam = cv2.VideoCapture(0)
font=cv2.FONT_HERSHEY_PLAIN
cv2.namedWindow("opencv_face", cv2.WINDOW_AUTOSIZE)



while True:
    ret,frame = cam.read()
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces_coord = detect_face(gray) # detect more than one face
    if len(faces_coord):
        faces = normalize_faces(frame, faces_coord)
        #faces = normalize_intensity(faces)
        for i, face in enumerate(faces): # for each detected face
            
            
            #cv2.imwrite('trainingData/female/picture_BGR5.jpg',face)
            t=face.reshape(1,-1)
            t=sc.transform(t.astype(np.float64))
            test = pca1.transform(t)    
            #print test
            #transform = test.reshape(1,-1)
            #print transform
            prob=svc1.predict_proba(test)
            confidence = svc1.decision_function(test)
            print (confidence)
            print (prob)
           
            
            
            pred = svc1.predict(test)
            print (pred,pred[0])
           
            name=labels_dic[pred[0]].capitalize()
            print (name)
            
            #pred = labels_dic[pred[0]].capitalize()
            #threshold = .50
            
            
                
            #cv2.putText(frame,name,(faces_coord[i][0], faces_coord[i][1] - 10),
             #          cv2.FONT_HERSHEY_PLAIN, 2, (66, 53, 243), 2)
            
                
           
            if prob[0][0]>.95:
                
                cv2.putText(frame, 'gopi',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 2, (66, 53, 243), 2)
            
                
            elif prob[0][1]<.9:
                cv2.putText(frame,'unknown',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
                
                
           
        clear_output(wait = True)
        draw_rectangle(frame, faces_coord) # rectangle around face
        
    cv2.putText(frame, "ESC to exit", (5, frame.shape[0] - 5),cv2.FONT_HERSHEY_PLAIN, 1.3, (66, 53, 243), 2,cv2.LINE_AA)
    
    cv2.imshow("opencv_face", frame) # live feed in external
    if cv2.waitKey(5) == 27:
        break
        
cam.release()
cv2.destroyAllWindows()
        

[0.51871342]
[[0.03390762 0.96609238]]
[1] 1
Gopi


In [77]:
cam = cv2.VideoCapture(0)
font=cv2.FONT_HERSHEY_PLAIN
cv2.namedWindow("opencv_face", cv2.WINDOW_AUTOSIZE)



while True:
    ret,frame = cam.read()
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces_coord = detect_face(gray) # detect more than one face
    if len(faces_coord):
        faces = normalize_faces(frame, faces_coord)
        #faces = normalize_intensity(faces)
        for i, face in enumerate(faces): # for each detected face
            
            
            #cv2.imwrite('trainingData/female/picture_BGR5.jpg',face)
            t=face.reshape(1,-1)
            t=sc.transform(t.astype(np.float64))
            test = pca1.transform(t)    
            #print test
            #transform = test.reshape(1,-1)
            #print transform
            prob=clf2.predict_proba(test)
            #confidence = svc1.decision_function(test)
            #print (confidence)
            print (prob)
           
            
            
            pred = clf2.predict(test)
            print (pred,pred[0])
           
            name=labels_dic[pred[0]].capitalize()
            print (name)
            
            
           
            if prob[0][0]>.65:
                
                cv2.putText(frame, 'gopi',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 2, (66, 53, 243), 2)
            
                
            elif prob[0][1]>.8:
                cv2.putText(frame,'darlinggopi',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
                
            elif prob[0][2]>.8:
                cv2.putText(frame,'unknown',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
                
                
        clear_output(wait = True)
        draw_rectangle(frame, faces_coord) # rectangle around face
        
    cv2.putText(frame, "ESC to exit", (5, frame.shape[0] - 5),cv2.FONT_HERSHEY_PLAIN, 1.3, (66, 53, 243), 2,cv2.LINE_AA)
    
    cv2.imshow("opencv_face", frame) # live feed in external
    if cv2.waitKey(5) == 27:
        break
        
cam.release()
cv2.destroyAllWindows()
        

[[0.17000366 0.82999634]]
[1] 1
Gopi


In [78]:
cam = cv2.VideoCapture(0)
font=cv2.FONT_HERSHEY_PLAIN
cv2.namedWindow("opencv_face", cv2.WINDOW_AUTOSIZE)



while True:
    ret,frame = cam.read()
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces_coord = detect_face(gray) # detect more than one face
    if len(faces_coord):
        faces = normalize_faces(frame, faces_coord)
        #faces = normalize_intensity(faces)
        for i, face in enumerate(faces): # for each detected face
            
            
            #cv2.imwrite('trainingData/female/picture_BGR5.jpg',face)
            t=face.reshape(1,-1)
            t=sc.transform(t.astype(np.float64))
            test = pca1.transform(t)    
            #print test
            #transform = test.reshape(1,-1)
            #print transform
            prob=clf2.predict_proba(test)
            #confidence = svc1.decision_function(test)
            #print (confidence)
            print (prob)
           
            
            
            pred = clf2.predict(test)
            print (pred,pred[0])
           
            name=labels_dic[pred[0]].capitalize()
            print (name)
            
            
            for j in label_dic:
                
                if prob[0][j]>.95:
                
                    cv2.putText(frame, k,(faces_coord[i][0], faces_coord[i][1] - 10),
                                cv2.FONT_HERSHEY_PLAIN, 2, (66, 53, 243), 2)
            
                
           
                
            
                
        clear_output(wait = True)
        draw_rectangle(frame, faces_coord) # rectangle around face
        
    cv2.putText(frame, "ESC to exit", (5, frame.shape[0] - 5),cv2.FONT_HERSHEY_PLAIN, 1.3, (66, 53, 243), 2,cv2.LINE_AA)
    
    cv2.imshow("opencv_face", frame) # live feed in external
    if cv2.waitKey(5) == 27:
        break
        
cam.release()
cv2.destroyAllWindows()
        

[[0.99815132 0.00184868]]
[0] 0
Darlinggopi


NameError: name 'label_dic' is not defined